# **INTRODUCTIONS**

---

The **boostershots** are additional vaccination shot when the effectiveness - regarding the first series of vaccinations - has been reduced and the **third vaccincation shot** are additional vaccination shot for immune sensitive people. Both vaccinations are represented on the **[vaccination page](https://coronadashboard.rijksoverheid.nl/landelijk/vaccinaties)**.   

The source data (i.e. **[RIVM](https://www.rivm.nl/en)**) are modelled on daily registrations.

The code is separated into multiple sections:

1. **[Views](#views)**
2. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/statics/safety_regions/*",
        "src/statics/municipalities/*"
    ]
}
```

# **TABLES**

---

### INTERMEDIATES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) CREATE INTER SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[dbo].[SEQ_VWSINTER_COVID19_MUNICIPALITY]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSINTER_COVID19_MUNICIPALITY]
  START WITH 1
  INCREMENT BY 1;
GO

-- 2) CREATE INTER TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[RIVM_COVID_19_NUMBER_MUNICIPALITY]'))
CREATE TABLE [VWSINTER].[RIVM_COVID_19_NUMBER_MUNICIPALITY] (
	[ID] [INT] PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSINTER_COVID19_MUNICIPALITY]),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[[DATE_OF_REPORT] [DATETIME] NULL,
	[DATE_OF_PUBLICATION] [DATETIME] NULL,
	[MUNICIPALITY_CODE] [VARCHAR](100) NULL,
	[MUNICIPALITY_NAME] [VARCHAR](100) NULL,
	[PROVINCE] [VARCHAR](100) NULL,
	[SECURITY_REGION_CODE] [VARCHAR](100) NULL,
	[SECURITY_REGION_NAME] [VARCHAR](100) NULL,
	[MUNICIPAL_HEALTH_SERVICE] [VARCHAR](100) NULL,
	[ROAZ_REGION] [VARCHAR](100) NULL,
	[TOTAL_REPORTED] [INT] NULL,
	[HOSPITAL_ADMISSION] [INT] NULL,
	[DECEASED] [INT] NULL
);
GO

-- 3) CREATE INTER INDEX(ES).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_INTER_RIVM_COVID_19_NUMBER_MUNICIPALITY')
CREATE NONCLUSTERED INDEX [IX_INTER_RIVM_COVID_19_NUMBER_MUNICIPALITY] ON [VWSINTER].[RIVM_COVID_19_NUMBER_MUNICIPALITY] (
    [DATE_LAST_INSERTED] ASC
) WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO

# **VIEWS**

---

In de view VWSReport.V\_Aandeel\_Positief\_Getest\_per\_Regio wordt gebruik gemaakt van de tabel VWSStatic.Safety\_Regions\_per\_Municipal. Hier staat de relatie in tussen gemeenten en veiligheidsregios. Om te verzekeren dat de meest recente dataset wordt gebruikt, is in de subquery waar deze tabel wordt gebruikt, toegevoegd dat de meest recente dataset gebruikt moet worden.

N.B. er moet dan wel een nieuwe versie van de tabel via de Static flow worden ingelezen.

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- THIS VIEW USES THE 'SAFETY_REGIONS_PER_MUNICIPAL' TO FETCH THE RELATIONS BETWEEN MUNICIPALITIES AND SAFETY REGIONS
CREATE OR ALTER VIEW [VWSREPORT].[V_AANDEEL_POSITIEF_GETEST_PER_REGIO] AS
-- SELECT BASE RECORDS
WITH BASE_CTE AS (
SELECT 
	[DATE_OF_PUBLICATION] AS [DATE_OF_REPORT]
,	[SECURITY_REGION_CODE] AS [VRCODE]
,	[MUNICIPALITY_CODE] AS [GMCODE]
,	[TOTAL_REPORTED]
,	[INHABITANTS]
,   T1.[DATE_LAST_INSERTED]
FROM [VWSINTER].[RIVM_COVID_19_NUMBER_MUNICIPALITY] T1
LEFT JOIN [VWSSTATIC].[INHABITANTS_PER_SAFETY_REGION] T3 ON [SECURITY_REGION_CODE] = T3.VGRNR
WHERE T1.[DATE_LAST_INSERTED] = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSINTER].[RIVM_COVID_19_NUMBER_MUNICIPALITY])
),
SECOND_CTE AS (
SELECT
    [DATE_OF_REPORT]
,   [VRCODE]
,   SUM([TOTAL_REPORTED]) AS [TOTAL_REPORTED]
,   SUM(CAST([TOTAL_REPORTED] AS FLOAT))/(CAST([INHABITANTS] AS DECIMAL(10,2))/CAST(100000 AS DECIMAL(10,2))) AS [REPORTED_PER_REGION_100000]
,   [INHABITANTS]
,   [DATE_LAST_INSERTED]
FROM BASE_CTE
GROUP BY [DATE_LAST_INSERTED], [DATE_OF_REPORT], [VRCODE], [INHABITANTS]
),
-- FINAL SELECT AND INSERT INTO STATEMENT. THIS PART ADDS THE CALCULATION OF A MOVING AVERAGE OF INCREASED HOSPITAL ADMISSIONS AND CUMULATIVE CALCULATIONS. IT CALCULATES IT BASED ON THE PREVIOUS 2 DAYS AS CAN BE SEEN IN ROW 59, AFTER BETWEEN.
THIRD_CTE AS  (
	SELECT
		T_SECOND.[DATE_OF_REPORT]
	,	T_SECOND.[VRCODE]
	,   ROUND((SUM([TOTAL_REPORTED]) OVER (PARTITION BY T_SECOND.[DATE_LAST_INSERTED], T_SECOND.[VRCODE] ORDER BY T_SECOND.[DATE_OF_REPORT] ASC)) / (CAST([INHABITANTS] AS FLOAT)/100000.0), 2) AS [DAGTOTAAL]
	,	ROUND([REPORTED_PER_REGION_100000], 1) AS [TOENAME_TOV_T-1]
	FROM SECOND_CTE AS T_SECOND
	WHERE T_SECOND.[DATE_OF_REPORT] BETWEEN ((SELECT MAX([DATE_OF_REPORT]) FROM SECOND_CTE) - DAY(6)) AND (SELECT MAX([DATE_OF_REPORT]) FROM SECOND_CTE)
	AND T_SECOND.[VRCODE] != ''
),
FOURTH_CTE AS (
	SELECT 
		[DATE_OF_REPORT],
		SUM([DAGTOTAAL]) AS [TOTAAL_WEEK]
	FROM
	(
		SELECT  DISTINCT A.*
		FROM THIRD_CTE A
		WHERE [DATE_OF_REPORT] = (SELECT MAX([DATE_OF_REPORT]) FROM THIRD_CTE)
	) A
	GROUP BY [DATE_OF_REPORT]
)
SELECT 
	A.[DATE_OF_REPORT],
	[VRCODE],
	[VRNAAM],
	(CAST([DAGTOTAAL] AS FLOAT)/B.[TOTAAL_WEEK]) * 100 [AANDEEL],
	[DAGTOTAAL],
	[TOENAME_TOV_T-1]
FROM (
	SELECT DISTINCT 
		A.*, 
		B.[VRNAAM]
	FROM THIRD_CTE A
	JOIN (
			SELECT * 
			FROM [VWSSTATIC].[SAFETY_REGIONS_PER_MUNICIPAL] 
			WHERE [DATE_LAST_INSERTED]=(SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[SAFETY_REGIONS_PER_MUNICIPAL])
		) B	ON A.[VRCODE] = B.[VRCODE]
	WHERE [DATE_OF_REPORT] = (SELECT MAX([DATE_OF_REPORT]) FROM THIRD_CTE)
) A, FOURTH_CTE B
GO